### Setup Configurations

In [2]:
import import_ipynb
import setup
import sys
import re
import pandas as pd

<h3 style="color:#6fda44;">Get form inputs for form of Type Task 1</h3>

(a)  a stock's ticker symbol  [OR PART OF THE COMPANY'S / ENTITY'S NAME] <br>
(b)  the form type needed to conduct the search  [OPTIONAL TO THE USER; THIS SEARCH WILL BE LIMITED TO 10-Ks AND 10-Qs AND THEIR VARIATIONS, e.g. 10-KSB, 10-K405, 10-QSB, etc.], <br>
(c)  the [FINANCIAL STATEMENT REPORT OR DISCLOSURE] information sought within the form <br>
(d)  optionally, the filing period <br>


*extract financial information from www.sec.gov, including items as categorized in filings (e.g. paragraph text AND TABLES in "Item 7" of Form 10-K or the balance sheet table in "Item 1" of Form 10-Q).* <br> <br>
Example A: 

*User input: (a) "AAPL" for Apple, Inc., (b) "10-K", and (c) "Balance Sheet", (d) left blank Output *
<br>
*The result should be the balance sheet table from the company's most recent 10-K filing* <br><br>
 **<span style="color:#6fda44;">[IF (b) AND (d) ARE NOT SPECIFIED BY USER, THEN THE MOST RECENT FINANCIAL STATEMENT OF (c) SHOULD BE RETRIEVED FROM FORMS 10-K OR 10-Q, WHICHEVER HAS THE LATER FILING DATE]</span>** <br><br>
Example B: 
 *User input: (a) "AAPL" for Apple, Inc., (b) "10-Q", (c) "Item 2", and (d) date of filing Output* <br>
*The result should be the text paragraphs from Item 2 of the company's 10-Q filing of that filing date* <br><br>
**<span style="color:#6fda44;">[LIKE THE COMMENT ABOVE, IF (d), FILING DATE, IS NOT SPECIFIED BY USER, THEN THE MOST RECENT USER-SPECIFIED "ITEM" IN THE USER SPECIFIED FORM SHOULD BE RETRIEVED]</span>** <br><br>


In [3]:
# Get form inputs
form_inputs = setup.get_inputs_tsk1()

Enter Stock Ticker Symbol>>  amazon
Enter Filing Type Required (10-K,10-Q,etc)>>  
Do you want to search for a Financial Table (T) or Particular Item in Document (I),Please Enter (T) or (I)>>  i
Do you want to get an item in Part I (1) or Part II (2),Please Enter (1) or (2)>>  1
Enter Item Number>> Item 1
To enter Date of Filing only enter one date. To enter Date period enter 2 dates.  
Filed FROM Date: (YYYY-MM-DD)>>  
Filed TO Date: (YYYY-MM-DD)>>  


In [4]:
#generate dynamic search url for queries
search_url = setup.generate_search_url(form_inputs)

#fetch all search results for query we need only one Result for task one
results = setup.get_search_results(form_inputs,1)

if results is None:
    print("Your Search did not yield any results",end="\n\n")
    # todo: end execution if results is none

SEARCH_URL https://www.sec.gov/edgar/search/#/q=%2522Item%25201%2522&category=custom&entityName=amazon&forms=10-K%252C10-Q&startdt=2019-03-03&enddt=2022-03-02

Total results are: 13

Exhibits Records (exe) Ignored: 1



In [5]:
# Check if user is searching for a particular financial Statement or Particular Item
if form_inputs.get('search_type','').lower() == 't' and results is not None:
    #     fetch table
    table_to_fetch = form_inputs.get('target','')
    # get links to pages
    links_dict = results.to_dict('records')[0]
    filing_summary_link = links_dict.get('filing_summa_xml_path')
    
    print("FILING SUMMARY LINK",filing_summary_link, end="\n\n")
    index_page = links_dict.get('index_page')
    soup = setup.get_generic_marketdata(filing_summary_link)
    tag = soup.find( lambda tag:tag.name.lower() == "shortname" and table_to_fetch.lower() in tag.text.lower() )
    
    if tag:
        parent = tag.parent
        table_link = index_page + '/' + parent.find('htmlfilename').text
        table_full_ttl = parent.find('longname').text
        soup = setup.get_generic_marketdata(table_link)
        table_MN = pd.read_html(str(soup.table))[0]

        print('FINANCIAL DATA TITLE:',table_full_ttl)
        print('TABLE LINK:',table_link, end="\n\n")

        table_MN
        
    else:
        print(f"Document with title: {table_to_fetch} Not found")


In [6]:
# If user was searching for a Particular Item in Document handler
if form_inputs.get('search_type','').lower() == 'i' and results is not None:
    
    print(
        f"Your searching for a particular Item in Part {form_inputs.get('item_part')} and Item number {form_inputs.get('target')}",
        end="\n\n"
    )
    print(form_inputs, end="\n\n")
    links_dict = results.to_dict('records')[0]
    print(links_dict)

Your searching for a particular Item in Part 1 and Item number Item 1

{'ticker': 'amazon', 'filing_type': '', 'search_type': 'i', 'item_part': '1', 'target': 'Item 1', 'filing_date_start': '', 'filing_date_end': ''}

{'file_date': '2022-02-04', 'filing_no': '000101872422000005', 'file_type': '10-K', 'file_name': 'amzn-20211231.htm', 'ticker': '0001018724', 'period_ending': '2021-12-31', 'display_names': 'AMAZON COM INC  (AMZN)  (CIK 0001018724)', 'filing_summa_xml_path': 'https://www.sec.gov/Archives/edgar/data/0001018724/000101872422000005/FilingSummary.xml', 'file_path': 'https://www.sec.gov/Archives/edgar/data/0001018724/000101872422000005/amzn-20211231.htm', 'index_page': 'https://www.sec.gov/Archives/edgar/data/0001018724/000101872422000005'}
